In [1]:
from rdflib import Graph, URIRef
import pandas as pd
from tqdm.notebook import tqdm
from collections import defaultdict

In [2]:
GRAPH_1 = "/Users/yamamotobikutorueiichi/codes/custom_paris/star-wars-wiki.xml"
GRAPH_1_INV_FUNC_PATH = '/Users/yamamotobikutorueiichi/codes/custom_paris/sww-inversability.csv'

GRAPH_2 = "https://dbpedia.org/sparql"
GRAPH_2_INV_FUNC_PATH = '/Users/yamamotobikutorueiichi/codes/gollumn - subdataset/sw-dbpedia/dbpedia-inversability.csv'

functionality_threshold = 0.8

In [3]:
graph_1 = Graph()
graph_1.parse(GRAPH_1)

#graph_2 = Graph()
#graph_2.parse(GRAPH_2)
graph_2 = Graph("SPARQLStore")
graph_2.open(GRAPH_2)

In [4]:
graph_1_inv_func_df = pd.read_csv(GRAPH_1_INV_FUNC_PATH)
graph_1_func_dict = dict()

for _, row in graph_1_inv_func_df.iterrows():
    graph_1_func_dict[row['predicate']] = row['functionality']

In [5]:
graph_2_inv_func_df = pd.read_csv(GRAPH_2_INV_FUNC_PATH)
graph_2_func_dict = dict()

for _, row in graph_2_inv_func_df.iterrows():
    graph_2_func_dict[row['predicate']] = row['functionality']

In [7]:
wikia = 'http://dbkwik.webdatacommons.org/starwars.wikia.com/resource'
dbpedia = 'http://dbkwik.webdatacommons.org/swg.wikia.com/resource'
match_list = list()
matched_o2_set = set()

for sub, pred, obj in graph:
    obj_text = str(obj)
    if wikia in obj_text:
        for s, p, o in graph.triples((sub, None, None)):
            o_text = str(o)
            if dbpedia in o_text:
                match_list.append({'e1': obj_text,
                                 'e2': o_text})
                matched_o2_set.add(o_text)
                break

In [6]:
entity_match_df = pd.read_csv('/Users/yamamotobikutorueiichi/codes/gollumn - subdataset/star-wars-dbpedia.csv')

In [7]:
match_list = entity_match_df.to_dict('records')

In [8]:
len(match_list)

2193

In [9]:
wikia = 'http://dbkwik.webdatacommons.org/starwars.wikia.com/property'
dbpedia = 'http://dbkwik.webdatacommons.org/swg.wikia.com/property'
property_match_list = list()

for sub, pred, obj in graph:
    obj_text = str(obj)
    if wikia in obj_text:
        for s, p, o in graph.triples((sub, None, None)):
            o_text = str(o)
            if dbpedia in o_text:
                property_match_list.append({'p1': obj_text,
                                 'p2': o_text})
                break

In [21]:
property_match_df = pd.read_csv('/Users/yamamotobikutorueiichi/codes/gollumn - subdataset/sw-dbpedia/sw-dbpedia/predicate_sem_prob.csv')


In [22]:
property_match_list = property_match_df.to_dict('records')

In [23]:
len(property_match_list)

736

In [25]:
filtered_prop_match_list = list()
for prop_match in property_match_list:
    p1 = prop_match['p1']
    p2 = prop_match['p2']
    if p1 not in graph_1_func_dict:
        continue
    
    if graph_1_func_dict[p1] > functionality_threshold and graph_2_func_dict[p2] > functionality_threshold:
        filtered_prop_match_list.append({
            'p1': p1,
            'p2': p2
        })

In [26]:
property_match_list = filtered_prop_match_list

In [27]:
len(property_match_list)

246

In [28]:
def get_predicates_objects(graph, subject):
    subj_uri = URIRef(subject)
    
    predicate_object_dict = dict()
    for p, o in graph.predicate_objects(subj_uri):
        predicate_object_dict.setdefault(str(p), []).append(str(o))
    return predicate_object_dict

In [29]:
def get_objects(graph, subject, predicate):
    subj_uri = URIRef(subject)
    pred_uri = URIRef(predicate)
    
    return list(graph.objects(subj_uri, pred_uri))

In [30]:
triple_matches = list()
for match in tqdm(match_list):
    predicate_objects_1 = get_predicates_objects(graph_1, match['e1'])
    predicate_objects_2 = get_predicates_objects(graph_2, match['e2'])

    for pred_match in property_match_list:
        if pred_match['p1'] not in predicate_objects_1 or pred_match['p2'] not in predicate_objects_2:
            continue
        objects_1 = predicate_objects_1[pred_match['p1']]
        objects_2 = predicate_objects_2[pred_match['p2']]
        
        for o1 in objects_1:
            for o2 in objects_2:
                triple_matches.append({
                    'e1': match['e1'],
                    'p1': pred_match['p1'],
                    'o1': o1,
                    'e2': match['e2'],
                    'p2': pred_match['p2'],
                    'o2': o2
                })

  0%|          | 0/2193 [00:00<?, ?it/s]

http://dbpedia.org/class/yago/WikicatMusicVideosDirectedBy"WeirdAl"Yankovic does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/class/yago/WikicatSongsWithLyricsBy"WeirdAl"Yankovic does not look like a valid URI, trying to serialize this will break.
http://dbpedia.org/class/yago/Wikicat"WeirdAl"YankovicSongs does not look like a valid URI, trying to serialize this will break.
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x103d6d310>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/opt/homebrew/Caskroom/miniforge/base/envs/berkeley/lib/python3.9/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)

In [31]:
match_dict = dict()

for match in match_list:
    match_dict[match['e1']] = match['e2']

In [32]:
def check_if_equals(o1, o2):
    o1_text = str(o1)
    o2_text = str(o2)
    if o1_text == o2_text:
        return True
    
    if type(o1) != URIRef or type(o2) != URIRef:
        return ''
    
    if o1_text in match_dict:
        if o2_text == match_dict[o1_text]:
            return True
        else:
            return False
    if o2_text in matched_o2_set:
        return False
    if type(o1) == URIRef and type(o2) == URIRef:
        return 'Unknown'
    
    return ''

In [33]:
len(triple_matches)

328

In [34]:
triple_matches_df = pd.DataFrame(triple_matches)

In [35]:
triple_matches_df['equals'] = triple_matches_df.apply(lambda x: check_if_equals(x['o1'], x['o2']), axis=1)

In [36]:
triple_matches_df

,e1,p1,o1,e2,p2,o2,equals
0,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,American,http://dbpedia.org/resource/Adam_Gidwitz,http://dbpedia.org/property/nationality,American,True
1,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,American,http://dbpedia.org/resource/Mike_W._Barr,http://dbpedia.org/property/nationality,American,True
2,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,http://www.claregrant.com,http://dbpedia.org/resource/Clare_Grant,http://dbpedia.org/property/website,http://www.claregrant.com,True
3,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbpedia.org/resource/Star_Wars_Jedi_Kni...,http://dbpedia.org/ontology/developer,http://dbpedia.org/resource/Raven_Software,
4,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,American,http://dbpedia.org/resource/Simon_Kinberg,http://dbpedia.org/property/nationality,American,True
...,...,...,...,...,...,...,...
323,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,Canadian,http://dbpedia.org/resource/Marvin_Goldhar,http://dbpedia.org/property/nationality,http://dbpedia.org/resource/Canadians,
324,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,Canadian,http://dbpedia.org/resource/Marvin_Goldhar,http://dbpedia.org/ontology/nationality,http://dbpedia.org/resource/Canadians,
325,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,American,http://dbpedia.org/resource/Nnedi_Okorafor,http://dbpedia.org/property/nationality,American,True
326,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,American,http://dbpedia.org/resource/Nnedi_Okorafor,http://dbpedia.org/property/nationality,Nigerian,


In [37]:
triple_matches_df[triple_matches_df['equals'] == '']

,e1,p1,o1,e2,p2,o2,equals
3,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbpedia.org/resource/Star_Wars_Jedi_Kni...,http://dbpedia.org/ontology/developer,http://dbpedia.org/resource/Raven_Software,
6,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbpedia.org/resource/Watto,http://dbpedia.org/property/gender,Male,
8,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,German-American,http://dbpedia.org/resource/Josef_Rubinstein,http://dbpedia.org/property/nationality,American,
9,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,http://www.robpaulsen.com/,http://dbpedia.org/resource/Rob_Paulsen,http://dbpedia.org/property/website,https://web.archive.org/web/20200811180903/htt...,
10,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbpedia.org/resource/Chott_el_Djerid,http://dbpedia.org/ontology/country,http://dbpedia.org/resource/Tunisia,
...,...,...,...,...,...,...,...
320,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,American,http://dbpedia.org/resource/Tom_Lyle,http://dbpedia.org/ontology/nationality,http://dbpedia.org/resource/Americans,
322,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,English-Argentine,http://dbpedia.org/resource/Olivia_Hussey,http://dbpedia.org/property/nationality,British,
323,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,Canadian,http://dbpedia.org/resource/Marvin_Goldhar,http://dbpedia.org/property/nationality,http://dbpedia.org/resource/Canadians,
324,http://dbkwik.webdatacommons.org/starwars.wiki...,http://dbkwik.webdatacommons.org/starwars.wiki...,Canadian,http://dbpedia.org/resource/Marvin_Goldhar,http://dbpedia.org/ontology/nationality,http://dbpedia.org/resource/Canadians,


In [38]:
triple_matches_df.to_csv('/Users/yamamotobikutorueiichi/codes/triple matching/sw-dbpedia-gold.csv')

In [31]:
triple_matches_df = pd.read_csv('/Users/yamamotobikutorueiichi/codes/triple matching/ma-mb-gold.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/yamamotobikutorueiichi/codes/triple matching/ma-mb-gold.csv'

In [133]:
triple_matches_df

,Unnamed: 0,e1,p1,o1,e2,p2,o2,equals
0,0,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Concord,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/status,Active,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Concord_(NCC-68711),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/status,active,TRUE
1,1,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Concord,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/name,USS Concord,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Concord_(NCC-68711),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/name,USS Concord,TRUE
2,2,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Gorkon,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/status,Active,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Gorkon_(NCC-40512),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/status,Active,TRUE
3,3,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Gorkon,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/name,USS Gorkon,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Gorkon_(NCC-40512),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/name,USS Gorkon,TRUE
4,4,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Defiant_class,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/type,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Escort_vessel,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Defiant_class,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/type,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Escort,TRUE
...,...,...,...,...,...,...,...,...
1855,1855,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Al-Batani,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/name,USS Al-Batani,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Al-Batani_(NCC-42995),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/name,USS Al-Batani,TRUE
1856,1856,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Potemkin_(Excelsior_class),http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/status,Active,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Potemkin_(NCC-18253),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/status,Active,TRUE
1857,1857,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Potemkin_(Excelsior_class),http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/name,USS Potemkin,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Potemkin_(NCC-18253),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/name,USS Potemkin,TRUE
1858,1858,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/New_Orleans_class,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/type,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Frigate,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/New_Orleans_class,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/type,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Frigate,TRUE


In [134]:
triple_matches_df.groupby(['p1', 'p2'])['equals'].value_counts().unstack(fill_value=0)

,equals,FALSE,TRUE,Unknown
p1,p2,,,
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/actor,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/actor,54,142,0
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/affiliation,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/affiliation,71,31,28
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/beam,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/beam,1,1,0
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/born,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/born,74,22,0
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/capital,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/capital,0,1,0
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/children,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/children,13,10,13
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/class,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/class,18,4,1
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/crew,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/crew,7,10,0
http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/date,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/date,7,12,0


In [135]:
pd.set_option('display.max_colwidth', None)


In [136]:
triple_matches_df[triple_matches_df['p1'] 
                  == 'http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/died'][['e1', 'e2',
                                                                                                    'o1', 'o2',
                                                                                                   'equals']]

,e1,e2,o1,o2,equals
18,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Garrovick_(Captain),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Steve_Garrovick,2257,2257,TRUE
275,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Walker_Keel,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Walker_Keel,2364,2364,TRUE
276,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Walker_Keel,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Walker_Keel,USS Horatio,2364,FALSE
320,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Michael_Eddington,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Michael_Eddington,2373,2373,TRUE
640,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Hawk_(Lieutenant),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Hawk_(Lieutenant),2063,2373,FALSE
686,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Charles_Tucker_III,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Charles_Tucker_III,2154,21612165,FALSE
687,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Charles_Tucker_III,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Charles_Tucker_III,2161,21612165,FALSE
726,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Lee_Kelso,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Lee_Kelso,2265,2265,TRUE
727,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Lee_Kelso,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Lee_Kelso,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Delta_Vega,2265,FALSE
764,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Sonak,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Sonak,2270,2273,TRUE


In [137]:
used_properties = ['http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/actor',
                  'http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/crew',
                  'http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/date',
                  'http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/defenses',
                  'http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/name',
                  'http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/rank',
                  'http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/species',
                  'http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/status']

In [138]:
final_dataset_df = triple_matches_df[triple_matches_df['p1'].isin(used_properties)]

In [139]:
final_dataset_df[final_dataset_df['equals'] == 'TRUE']

,Unnamed: 0,e1,p1,o1,e2,p2,o2,equals
0,0,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Concord,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/status,Active,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Concord_(NCC-68711),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/status,active,TRUE
1,1,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Concord,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/name,USS Concord,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Concord_(NCC-68711),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/name,USS Concord,TRUE
2,2,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Gorkon,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/status,Active,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Gorkon_(NCC-40512),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/status,Active,TRUE
3,3,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Gorkon,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/name,USS Gorkon,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Gorkon_(NCC-40512),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/name,USS Gorkon,TRUE
7,7,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Defiant_class,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/defenses,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/Cloaking_device,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Defiant_class,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/defense,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/Cloaking_device,TRUE
...,...,...,...,...,...,...,...,...
1853,1853,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Lakota,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/name,USS Lakota,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Lakota_(NCC-42768),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/name,USS Lakota,TRUE
1854,1854,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Al-Batani,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/status,Active,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Al-Batani_(NCC-42995),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/status,Active,TRUE
1855,1855,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Al-Batani,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/name,USS Al-Batani,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Al-Batani_(NCC-42995),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/name,USS Al-Batani,TRUE
1856,1856,http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/resource/USS_Potemkin_(Excelsior_class),http://dbkwik.webdatacommons.org/memory-alpha.wikia.com/property/status,Active,http://dbkwik.webdatacommons.org/stexpanded.wikia.com/resource/USS_Potemkin_(NCC-18253),http://dbkwik.webdatacommons.org/stexpanded.wikia.com/property/status,Active,TRUE


In [140]:
final_dataset_df.to_csv('')